In [1]:
#load "Paket.fsx"
Paket.Package
  [ "QBitNinja.Client -Version 1.0.3.21"
    "NBitcoin -Version 3.0.0.33"
    "FSharp.Data -Version 2.4.6"
    "Newtonsoft.Json -Version 10.0.3"    
  ]    
#load "Paket.Generated.Refs.fsx"

open System
open NBitcoin
open NBitcoin.Crypto  

In [5]:
type Wallet =
    static member PrivateKey: Key = 
        (new Key())
    static member PublicKey (k:Key) = 
        if (k=null) then
          (new Key()).PubKey
        else
          k.PubKey 
          
type Wallet with
     static member BitcoinSecretMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.Main)
            else
                k.GetBitcoinSecret(Network.Main)
     static member BitcoinSecretTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetBitcoinSecret(Network.TestNet)
            else
                k.GetBitcoinSecret(Network.TestNet)    
     static member WifMain (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.Main)
            else
                k.GetWif(Network.Main)
     static member WifTest (k:Key) = 
            if (k=null) then
                Wallet.PrivateKey.GetWif(Network.TestNet)
            else
                k.GetWif(Network.TestNet)
type Wallet with
     static member BitCoinAddressMain (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.Main) 
     static member BitCoinAddressTest  (k:Key) = 
        Wallet.PublicKey(k).GetAddress(Network.TestNet) 
     static member PublicKeyHash (k:Key) = 
        Wallet.PublicKey(k).Hash 
     static member BitCoinAddressMain2 (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.Main) 
     static member BitCoinAddressTest2  (k:Key) = 
        Wallet.PublicKey(k).Hash.GetAddress(Network.TestNet) 
        
type Script = 
   static member PublicKeyHash (Pk:string) = 
       new KeyId(Pk) 
   static member Main  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.Main).ScriptPubKey
   static member Test  (Pk:string) = 
       Script.PublicKeyHash(Pk).GetAddress(Network.TestNet).ScriptPubKey 
   static member Main  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.Main).ScriptPubKey 
   static member Test  (k:Key)  = 
       Wallet.PublicKeyHash(k).GetAddress(Network.TestNet).ScriptPubKey 
   static member BitCoinAddressMain (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.Main)
   static member BitCoinAddressTest  (k:Key) = 
      Script.Main(k).GetDestinationAddress(Network.TestNet)

In [2]:
open FSharp.Data
open FSharp.Data.JsonExtensions

type secretJson = JsonProvider<"""{
  "secret": ""
}""">
      
let sj = secretJson.GetSample()
//printfn "%A" sj 
  
let json_uri = @"/home/nbuser/library/secret_dest.json"
let wwif = secretJson.Load(json_uri).JsonValue?secret.AsString()  

let network = Network.TestNet

let dest_bitcoinPrivateKey  = Key.Parse(wwif, Network.TestNet) 
let dest_address = Script.BitCoinAddressTest dest_bitcoinPrivateKey //Wallet.PublicKeyHash bitcoinPrivateKey

printfn "%A" (network)
printfn "dest %A" (dest_address)
printfn "script dest %A" (Script.Test dest_bitcoinPrivateKey)

The field, constructor or member 'BitCoinAddressTest' is not defined.
The field, constructor or member 'Test' is not defined.

In [10]:
open QBitNinja.Client 
open QBitNinja.Client.Models 
open System.Collections.Generic 

let client:QBitNinjaClient  = new QBitNinjaClient(Network.TestNet)
let transactionId = uint256.Parse("06927bb3aaa2d1800548105d97a4f40ff61a68821f249a90158ddb76b082a7e8") 
let transactionResponse:GetTransactionResponse = client.GetTransaction(transactionId).Result 

printfn "%A" transactionResponse.TransactionId // 06927bb3aaa2d1800548105d97a4f40ff61a68821f249a90158ddb76b082a7e8
printfn "%A" transactionResponse.Block.Confirmations  // 190


let transaction:NBitcoin.Transaction  = transactionResponse.Transaction

printfn "%A" transaction

06927bb3aaa2d1800548105d97a4f40ff61a68821f249a90158ddb76b082a7e8
1378
{
  "hash": "06927bb3aaa2d1800548105d97a4f40ff61a68821f249a90158ddb76b082a7e8",
  "ver": 1,
  "vin_sz": 1,
  "vout_sz": 3,
  "lock_time": 0,
  "size": 268,
  "in": [
    {
      "prev_out": {
        "hash": "d5e087d429a1a42b3d42f5ce717621583988f8bf469f64afdff2c6060dab692b",
        "n": 0
      },
      "scriptSig": "3044022078cf72050fd3264fb4a264352105070f5112184b9aac6625ac58b07b3da68536022044148bf20abd1abae42b2368dbaa19267669bdcf32c83020457dca610830950201 0364dfe2fa8c584d9af693fca129f9c031e4108be2b6d31ac3f6846ef1ba7ecdc1"
    }
  ],
  "out": [
    {
      "value": "0.52980000",
      "scriptPubKey": "OP_DUP OP_HASH160 4fbd5152456bee3dc0ffe25118a6de17126e5c19 OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.01000000",
      "scriptPubKey": "OP_DUP OP_HASH160 8efb2874941ae0a85e2c94ee6b267e8ad70fb5cf OP_EQUALVERIFY OP_CHECKSIG"
    },
    {
      "value": "0.00000000",
      "scriptPubKey": "OP_RETURN 4bd03

In [13]:
let receivedCoins = transactionResponse.ReceivedCoins;
let mutable myCoins:Coin = null 
for coin in receivedCoins do
    if (coin.TxOut.ScriptPubKey = dest_bitcoinPrivateKey.ScriptPubKey) then
        myCoins<- coin:?>Coin

printfn "%A" myCoins.Amount

for coin in receivedCoins do
       printfn "%A" (coin.TxOut.get_ScriptPubKey().IsUnspendable)

The value, namespace, type or module 'dest_bitcoinPrivateKey' is not defined.

In [14]:
for coin in receivedCoins do
       printfn "%A" (coin.TxOut.get_ScriptPubKey().ToString())

The value or constructor 'receivedCoins' is not defined.
Lookup on object of indeterminate type based on information prior to this program point. A type annotation may be needed prior to this program point to constrain the type of the object. This may allow the lookup to be resolved.

In [6]:
let hash:string = "4bd0372fc54d125900f2068006a96eaa716a4919e5d842f5c591c20b2175625d"

In [10]:
open System
open System.IO
open System.Security.Cryptography

let  GetStream (s:string) =
                let stream = new MemoryStream()
                let writer = new StreamWriter(stream)
                writer.Write(s)
                writer.Flush()
                stream.Position <- 0L
                stream
                
//Print the byte array in a readable format.
let ByteString (b:byte[]) = 
        let mutable i = 0
        let mutable s:string = ""
        for i = 0  to b.Length - 1 do
            s <- s + String.Format("{0:X2}", b.[i])
            if (i % 4 = 3) then
                s <- s+ " "
        s    
        
let ByteToHex bytes = 
    bytes 
    |> Array.map (fun (x : byte) -> System.String.Format("{0:X2}", x))
    |> String.concat System.String.Empty
    
    
let Sha256 (doc:string) = 
    let sdoc = GetStream doc
    let sh = new SHA256Managed()
    sh.ComputeHash(sdoc)

let hash' = (ByteToHex((Sha256 "SF5128165575"))).ToString().ToLower()

printfn "%A" (hash')    

"4bd0372fc54d125900f2068006a96eaa716a4919e5d842f5c591c20b2175625d"


In [18]:
let vv =  String.Equals(hash, hash',StringComparison.CurrentCultureIgnoreCase)
printfn "%A" (vv) 

true
